In [38]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline

In [39]:
import pennylane as qml
from pennylane import numpy as np

np.random.seed(42)

In [40]:
num_qubits = 4
num_layers = 2
num_samples = 200
dev = qml.device("default.qubit", wires=num_qubits, shots=1000)
dev_gadget = qml.device("default.qubit", wires=2*num_qubits, shots=1000)

In [41]:
def hardware_efficient_ansatz(params):
    """A random variational quantum circuit based on the hardware efficient ansatz. 
    There are no measurements and it is to be used within the global or local circuits

    Args:
        params (array[array[float]]): array of parameters of dimension (num_layers, num_qubits) containing the rotation angles

    Returns:
        not sure... see pennylane documentation
    """

    # Relevant parameters
    assert(len(np.shape(params)) == 2)      # check proper dimensions of params
    num_layers = np.shape(params)[0]        # np.shape(params) = (num_layers, num_qubits)
    num_qubits = np.shape(params)[1]
    
    # Generating the gate sequence from randomly applying RX, RY or RZ with the corresponding rotation angle
    gate_set = [qml.RX, qml.RY, qml.RZ]
    random_gate_sequence = [[np.random.choice(gate_set) for _ in range(num_qubits)] for _ in range(num_layers)]

    # Initial rotations on all qubits
    for i in range(num_qubits):             # rotate all qubits
        qml.RY(np.pi / 4, wires=i)          # "to prevent X, Y , or Z from being an especially preferential 
                                            # direction with respect to gradients."

    # Repeating a layer structure
    for l in range(num_layers):
        # Single random gate layer (single qubit rotations)
        for i in range(num_qubits):
            random_gate_sequence[l][i](params[l][i], wires=i)
        # Nearest neighbour controlled phase gates
        if num_qubits > 1:                          # no entangling gates if using a single qubit
            qml.broadcast(qml.CZ, wires=range(num_qubits), pattern="ring")

In [42]:
@qml.qnode(dev)
def global_circuit(params):
    assert(len(np.shape(params)) == 2)      # check proper dimensions of params
    num_qubits = np.shape(params)[1]        # np.shape(params) = (num_layers, num_qubits)

    hardware_efficient_ansatz(params)
    # Objective operator H = Z_1 Z_2 ... Z_n
    H = qml.PauliZ(0)
    for qubit in range(num_qubits-1):
        H = H @ qml.PauliZ(qubit + 1)
    return qml.expval(H)

@qml.qnode(dev)
def local_circuit(params):
    hardware_efficient_ansatz(params)
    # Objective operator H = Z_1 Z_2
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))
 
@qml.qnode(dev_gadget)
def gadget_circuit(params, term, target_qubits):
    assert(len(np.shape(params)) == 2)
    computational_qubits = np.shape(params)[1]

    hardware_efficient_ansatz(params)

    # creating the "unperturbed Hamiltonian": Hanc
    if term == 'ancillary':
        # terms of the form I-ZZ
        term = qml.Identity(target_qubits[0]) @ qml.Identity(target_qubits[1]) - qml.PauliZ(target_qubits[0]) @ qml.PauliZ(target_qubits[1])
    elif term == 'coupling':
        # terms of the form ZxX
        term = qml.PauliZ(target_qubits[0]) @ qml.PauliZ(computational_qubits+target_qubits[0])
    return qml.expval(term)

In [43]:
def cost_global(params):
    return global_circuit(params)

def cost_local(params):
    return local_circuit(params)

def cost_gadgets(params, lam=1):
    # Objective operator H = Hanc + V
    computational_qubits = num_qubits
    ancillary_qubits = computational_qubits
    total_qubits = ancillary_qubits + computational_qubits
    expval_terms = []
    # creating the "unperturbed Hamiltonian"
    for first_qubit in range(computational_qubits, total_qubits):
        for second_qubit in range(first_qubit+1, computational_qubits+ancillary_qubits):
            expval_terms.append(gadget_circuit(params, 'ancillary', [first_qubit, second_qubit]))
    # creating the perturbation part of the Hamiltonian
    for qubit in range(computational_qubits):
        expval_terms.append(gadget_circuit(params, 'coupling', [qubit]))
    
    return np.sum(expval_terms[:-num_qubits]) + lam*np.sum(expval_terms[-num_qubits:])

In [45]:
params = np.random.uniform(0, np.pi, size=(num_layers, num_qubits))
print(cost_local(params))
print(cost_global(params))
print(cost_gadgets(params))
print(qml.grad(cost_local)(params)[0][0])
print(qml.grad(cost_global)(params)[0][0])
print(qml.grad(cost_gadgets)(params)[0][0])

0.5
0.052
-1.206
-0.16799999999999998
-0.021000000000000005
-0.1609999999999999
